In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import pandas as pd
import seaborn as sns
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.darts import DartsXGBModel
from sktime.split import (
    temporal_train_test_split,
)

from probafcst.backtest import backtest
from probafcst.plotting import plot_quantiles

sns.set_theme(style="ticks")

In [ ]:
quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")
load = load.asfreq("h")

In [ ]:
from sktime.forecasting.darts import DartsLinearRegressionModel
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss

forecaster = DartsLinearRegressionModel(
    lags=[-7, -30, -365],
    lags_future_covariates=[-7, -30, -365, *list(range(30))],
    add_encoders={"cyclic": {"future": ["day", "month", "day_of_year", "quarter"]}},
    output_chunk_length=7,
    quantiles=quantile_levels,
    likelihood="quantile",
    multi_models=False,
)

y_train, y_test = temporal_train_test_split(bikes, test_size=365)
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster.fit(y_train)
pred_quantiles = forecaster.predict_quantiles(fh, alpha=quantile_levels)
PinballLoss()(y_test, pred_quantiles)

In [ ]:
backtest(
    forecaster,
    bikes,
    forecast_steps=30,
    quantiles=quantile_levels,
    initial_window=365 * 3,
    step_length=90,
    backend="loky",
)

In [ ]:
y_train, y_test = temporal_train_test_split(
    load.iloc[-24 * 365 * 3 :], test_size=24 * 14
)
fh = ForecastingHorizon(y_test.index, is_relative=False)
output_chunk_length = 1  # len(splitter.fh)
forecaster = DartsXGBModel(
    lags=[-24],
    likelihood="quantile",
    quantiles=quantile_levels,
    multi_models=False,
    output_chunk_length=output_chunk_length,
)
forecaster.fit(y_train)

In [ ]:
pred_quantiles = forecaster.predict_quantiles(fh, alpha=quantile_levels)
plot_quantiles(y_test, pred_quantiles)

## Backtesting

In [ ]:
output_chunk_length = 24 * 14
add_encoders = {
    "cyclic": {"future": ["hour", "day", "month", "day_of_year"]},
}
forecaster = DartsXGBModel(
    lags=24 * 7,
    likelihood="quantile",
    quantiles=quantile_levels,
    multi_models=False,
    add_encoders=add_encoders,
    output_chunk_length=output_chunk_length,
    lags_future_covariates=[-24 * 30, -24 * 7, -24, 0, 24, 24 * 7, 24 * 30],
    kwargs={"n_jobs": 1},
)

In [ ]:
forecaster

In [ ]:
results, metrics, predictions, add_metrics = backtest(
    forecaster,
    y=load,
    forecast_steps=24 * 14,
    quantiles=quantile_levels,
    initial_window=24 * 365 * 3,
    step_length=24 * 30 * 3,
    backend="loky",
)

In [ ]:
results

In [ ]:
sns.set_theme(style="ticks")

In [ ]:
# create box plots for each quantile loss using results frame
# use melt for this
melted = results[quantile_levels].melt(var_name="quantile", value_name="loss")
melted["quantile"] = melted["quantile"].apply(lambda x: f"q{x}")
ax = sns.boxplot(data=melted, x="quantile", y="loss", hue="quantile")

In [ ]:
metrics

In [ ]:
# plot each forecast period
nrows = min(3, len(results))
for i, row in predictions.iloc[-nrows:].iterrows():
    plot_quantiles(row.y_test, row.y_pred_quantiles)